In this notebook we are getting the sentiment analysis for each review text for each product per
year <br>(taking into consideration only the product with high number of votes)

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 41.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c400fcaa7b339b5cc88fcbc32f5d63d9bc64f0192748675925990b8de273f83b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark import SparkContext
from itertools import product
import json
import re
from datetime import datetime
import numpy as np

In [ ]:
sample_meta = '/content/drive/Shareddrives/CIE/BigData_Final_Project/Data/meta'
sample = '/content/drive/Shareddrives/CIE/BigData_Final_Project/Data/reviews'

In [ ]:
# create Spark context with necessary configuration
sc = SparkContext("local","FinalProject")

In [ ]:
data_meta = sc.textFile(sample_meta)
data_review = sc.textFile(sample)

# Sentiment


In [ ]:
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 16.2 MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()

In [ ]:
def senAnalysis(txt):
  sen_dict = sentiments.polarity_scores(txt)
  if sen_dict['compound'] >= 0.05 :
    return 3    #"Positive"
  elif sen_dict['compound'] <= - 0.05 :
    return 2    #"Negative"
  else :
    return 1    #"Neutral"


In [ ]:
def product_rating(line):
  line = json.loads(line)
  return (line['asin'], line['overall'])

In [ ]:
def product_time_SenAnalysis(line):
  line = json.loads(line)
  time = datetime.fromtimestamp(line['unixReviewTime'])
  year = time.year
  try:
    senAna = senAnalysis(line['reviewText']) 
    return (line['asin'], (year,senAna))
  except:
    return (line['asin'], (year,4))

In [ ]:
#test
product_time_SenAnalysis(y[0])

('0060009810', (2002, 3))

In [ ]:
product_time_seAna = data_review.map(product_time_SenAnalysis).distinct()
product_time_seAna.take(5)

In [ ]:
productCount = data_review.map(product_rating).groupByKey()
productCount = productCount.map(lambda x: (x[0], len(list(x[1]))))
productCount.take(4)

In [ ]:
productRating = data_review.map(product_rating).reduceByKey(lambda a,b:a +b).sortByKey()
productAvgRating_ = productRating.join(productCount)
productAvgRating_.take(5)

In [ ]:
productAvgRating = productAvgRating_.map(lambda x: (x[0], (x[1][1],round(x[1][0]/x[1][1],2)))).sortByKey()
productAvgRating.take(5)

In [ ]:
productAvgRatingTimeSeAna = productAvgRating.join(product_time_seAna)
Sorted = productAvgRatingTimeSeAna.sortByKey()
Sorted.take(5)

In [ ]:
productAvgRatingTimeSeAna_ = Sorted.map(lambda x: (x[0], x[1][0][0],(x[1][0][1],x[1][1][0],x[1][1][1])))
productAvgRatingTimeSeAna_.take(5)

#### Modify Calculations

In [ ]:
max = productAvgRatingTimeSeAna_.max()[1]
min = productAvgRatingTimeSeAna_.min()[1]
part = (max+min)/4
part

### Modified Product_No.ofVotes_AvgRate

In [ ]:
modified_productAvgRatingTimeSeAna_ = productAvgRatingTimeSeAna_.filter(lambda x: x[1] > part).sortBy(lambda x: x[1], ascending=False)
modified_productAvgRatingTimeSeAna_.take(5)

### groupBy(productID) 
productID, year, sentimentAnalysis(ReviewText)

In [ ]:
byProduct = modified_productAvgRatingTimeSeAna_.groupBy(lambda x: x[0])
byProduct.take(5)

In [ ]:
def year_Avg_Rate_SeAna(line):
  return (line[2][1],(line[2][0],line[2][2]))

In [ ]:
year_Avg_Rate_SeAna(list(byProduct.take(5)[0][1])[0])

In [ ]:
def year_senAna(line):
  r = [x[1][1] for x in list(line[1])]
  sen_ana = np.argmax([r.count(1), r.count(2), r.count(3)])
  if sen_ana == 0:
    sen_return = 'Neutral'
  elif sen_ana == 1:
    sen_return = 'Negative'
  else: 
    sen_return = 'Positive'
  year_return = line[0]

  return (year_return, sen_return)

def year_sen(line): 
  RDD = sc.parallelize(list(line[1]))
  RDD_year = RDD.map(year_Avg_Rate_SeAna)
  RDD_year_ = RDD_year.groupBy(lambda x: x[0])
  xxx = RDD_year_.map(year_senAna)
  return (line[0],xxx.collect())

In [ ]:
year_sen(byProduct.take(6)[2])

In [ ]:
RandomSample_5Product = byProduct.sample(False,5)
Samples = RandomSample_5Product.collect()

for i in range(5):
  print(year_sen(Samples[i]))